In [1]:
import pandas as pd
import json

In [2]:
with open('../data/data_salto/concursos/concursos_salto_nacional_2025.json', 'r', encoding='utf-8') as file:
    datos_concursos = json.load(file)

df_concursos = pd.DataFrame(datos_concursos)
df_concursos = df_concursos.rename(columns = {"Nombre": "Concurso", "Categoría": "Categoria_concurso"})
df_concursos["Concurso"] = df_concursos["Concurso"].str.strip()
df_concursos.head()

,Concurso,Categoria_concurso,Provincia,Localidad,Disciplina,Federación,Resultados,País,Inicio,Final,Ambito
0,CSN3* Invierno,CSN***,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Ver,Epaña,10/01/2025,12/01/2025,Nacional
1,SPRING MET I 2025 YH*,CSI YH*,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,14/01/2025,16/01/2025,Internacional
2,SPRING MET I 2025 - SILVER 1*,CSI*,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,16/01/2025,18/01/2025,Internacional
3,SPRING MET I 2025 - GOLD 2*,CSI**,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,16/01/2025,19/01/2025,Internacional
4,CSIYH* - 1st Week Andalucía Pre-Sunshine Tour ...,CSI YH*,Cádiz,Dehesa Montenmedio,Salto de Obstáculos,Federación Hípica Andaluza,Ver,Epaña,16/01/2025,19/01/2025,Internacional


In [3]:
df_pruebas_resultados = pd.read_csv("../data/data_salto/resultados_combinados_2025.csv", index_col=0)
df_pruebas_resultados.head()

C:\Users\gabri\AppData\Local\Temp\ipykernel_13084\1077302799.py:1: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pruebas_resultados = pd.read_csv("../data/data_salto/resultados_combinados_2025.csv", index_col=0)


,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,...,Categoria,Ptos_obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_obs_R3,Ptos_tiempo_R3,Tiempo_R3
0,FIN,1,284725,Pedro Miguel DA SILVA MANSO,ESP,169882,BRIDGETOWN JONES OS,(Indt.),ESP,"0/59,91",...,CSN***,0.0,NaN,59.91,NaN,NaN,NaN,NaN,NaN,NaN
1,FIN,2,77612,Ignacio ASTOLFI LÓPEZ,ESP,157373,QUINTO BLUE PS,OLDBG,ESP,"0/63,23",...,CSN***,0.0,NaN,63.23,NaN,NaN,NaN,NaN,NaN,NaN
2,FIN,3,306183,Francisco PICAO CALDEIRA,ESP,145996,FLOR DU PASO,CRPOR,ESP,"0/65,58",...,CSN***,0.0,NaN,65.58,NaN,NaN,NaN,NaN,NaN,NaN
3,FIN,4,1004,Jesús TORRES GARCÍA,ESP,170075,DIABETTO PS,OLDBG,ESP,"0/66,00",...,CSN***,0.0,NaN,66.00,NaN,NaN,NaN,NaN,NaN,NaN
4,FIN,5,367363,Nina FAGERSTROM,ESP,170155,HASENACHER CERVIN,ZANG,ESP,"0/66,78",...,CSN***,0.0,NaN,66.78,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
& (df_final["Categoria"] == df_final["Categoria_concurso"])

In [15]:
df_final = pd.merge(df_pruebas_resultados, df_concursos, on = "Concurso", how = "inner")
df_final['Fecha_prueba'] = pd.to_datetime(df_final['Fecha_prueba'])
df_final['Inicio'] = pd.to_datetime(df_final['Inicio'], dayfirst=True)
df_final['Final'] = pd.to_datetime(df_final['Final'], dayfirst=True)
df_final = df_final[(df_final['Fecha_prueba'] >= df_final['Inicio']) & (df_final['Fecha_prueba'] <= df_final['Final'])]
df_final = df_final.drop(['Puntuacion', 'Resultados', 'Categoria_concurso', 'Pais_jinete', 'Pais_caballo', 'Raza_caballo'], axis=1)
df_final["Jinete"] = df_final["Jinete"].str.lower()
df_final["Caballo"] = df_final["Caballo"].str.lower()
df_final["Lic_jinete"] = df_final["Lic_jinete"].astype(str)
df_final["Lic_caballo"] = df_final["Lic_caballo"].astype(str)
# df_final["Raza_caballo"] = df_final["Raza_caballo"].apply(lambda x: None if x == '(Indt.)' else x)
df_final["Lic_jinete"] = df_final["Lic_jinete"].apply(lambda x: "10186822" if x == '--' else x)
df_final["Jinete"] = df_final["Jinete"].apply(lambda x: "jesus folch redondo" if x == '-- -- --' else x)
lista = ["ELI", "RET", "NOP"]
df_final["Posicion"] = df_final["Posicion"].apply(lambda x: None if x in lista else x)
df_final["Lic_caballo"] = df_final["Lic_caballo"].apply(lambda x: None if x == '--' else x)

lic_mas_larga_por_jinete = (
    df_final[["Jinete", "Lic_jinete"]]
    .drop_duplicates()
    .assign(longitud=lambda x: x["Lic_jinete"].astype(str).str.len())
    .sort_values("longitud", ascending=False)
    .drop_duplicates(subset="Jinete")
    .drop(columns="longitud")
    .set_index("Jinete")
)

lic_mas_larga_por_caballo = (
    df_final[["Caballo", "Lic_caballo"]]
    .drop_duplicates()
    .assign(longitud=lambda x: x["Lic_caballo"].astype(str).str.len())
    .sort_values("longitud", ascending=False)
    .drop_duplicates(subset="Caballo")
    .drop(columns="longitud")
    .set_index("Caballo")
)

# 2. Reemplazar en el DataFrame original
df_final["Lic_jinete"] = df_final["Jinete"].map(lic_mas_larga_por_jinete["Lic_jinete"])
df_final["Lic_caballo"] = df_final["Caballo"].map(lic_mas_larga_por_caballo["Lic_caballo"])
df_final


,Estado,Posicion,Lic_jinete,Jinete,Lic_caballo,Caballo,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Categoria,Ptos_obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_obs_R3,Ptos_tiempo_R3,Tiempo_R3,Provincia,Localidad,Disciplina,Federación,País,Inicio,Final,Ambito
0,FIN,1,284725,pedro miguel da silva manso,169882,bridgetown jones os,True,450.0,"1,30 - A c.c. (238.2.1)",2025-01-10,CSN3* Invierno,CSN***,0.0,NaN,59.91,NaN,NaN,NaN,NaN,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
1,FIN,2,77612,ignacio astolfi lópez,157373,quinto blue ps,True,300.0,"1,30 - A c.c. (238.2.1)",2025-01-10,CSN3* Invierno,CSN***,0.0,NaN,63.23,NaN,NaN,NaN,NaN,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
2,FIN,3,306183,francisco picao caldeira,145996,flor du paso,True,200.0,"1,30 - A c.c. (238.2.1)",2025-01-10,CSN3* Invierno,CSN***,0.0,NaN,65.58,NaN,NaN,NaN,NaN,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
3,FIN,4,10006303,jesús torres garcía,107ZP87,diabetto ps,True,150.0,"1,30 - A c.c. (238.2.1)",2025-01-10,CSN3* Invierno,CSN***,0.0,NaN,66.00,NaN,NaN,NaN,NaN,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
4,FIN,5,10002400,nina fagerstrom,105ZA09,hasenacher cervin,True,125.0,"1,30 - A c.c. (238.2.1)",2025-01-10,CSN3* Invierno,CSN***,0.0,NaN,66.78,NaN,NaN,NaN,NaN,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194685,FIN,23,77612,ignacio astolfi lópez,158640,grupo prom jivaro,True,0.0,"1,35 - Dos fases Especial (Art. 274.2) (274.2)",2025-01-10,CSN3* Invierno,CSN***,0.0,NaN,NaN,12.0,9.0,48.62,NaN,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
194686,FIN,24,37013,alejandro alonso buzo,159094,houston du gast,True,0.0,"1,35 - Dos fases Especial (Art. 274.2) (274.2)",2025-01-10,CSN3* Invierno,CSN***,8.0,17.0,NaN,0.0,2.0,41.45,NaN,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
194687,RET,None,10047790,patricio de valdenebro del rey,106IJ31,klimax van het lindehof,True,0.0,"1,35 - Dos fases Especial (Art. 274.2) (274.2)",2025-01-10,CSN3* Invierno,CSN***,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
194688,NOP,None,196126,francisco román cantillana,155590,uloubet de vanau,True,0.0,"1,35 - Dos fases Especial (Art. 274.2) (274.2)",2025-01-10,CSN3* Invierno,CSN***,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional


In [5]:
df_final.to_csv("../data/data_salto/informacion_concursos_final.csv")

NameError: name 'df_final' is not defined

In [2]:
df = pd.read_csv("../data/data_salto/informacion_concursos_final.csv", index_col = 0)
df["Disciplina"] = "Salto de Obstaculos"
i = 0
df["id_resultado"] = range(i +1 , i + 1 + len(df))
print(df["Lic_caballo"].duplicated().sum())
print(df["Caballo"].duplicated().sum())

64399
64399


C:\Users\gabri\AppData\Local\Temp\ipykernel_9052\2217570652.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/data_salto/informacion_concursos_final.csv", index_col = 0)


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df[df["Concurso"] == "MOURATOURS"]

,Estado,Posicion,Lic_jinete,Jinete,Lic_caballo,Caballo,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Categoria,Ptos_obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_obs_R3,Ptos_tiempo_R3,Tiempo_R3,Provincia,Localidad,Disciplina,Federación,País,Inicio,Final,Ambito,id_resultado
188620,FIN,1.0,2659,katia bosque izquierdo,168410,j'aime cassendra,True,49.0,"5 años - 1,10 - Dos Fases Especial (274.2)",2025-04-18,MOURATOURS,CSNCJ,NaN,0.0,NaN,0.0,NaN,44.12,NaN,NaN,NaN,Valencia,Mouratours Valencia,Salto de Obstaculos,Federación Hípica Comunidad Valenciana,Epaña,2025-04-18,2025-04-20,Nacional,68635
188621,FIN,2.0,46311,miguel laiz zandio,166951,poker face lz,True,35.0,"5 años - 1,10 - Dos Fases Especial (274.2)",2025-04-18,MOURATOURS,CSNCJ,NaN,0.0,NaN,0.0,NaN,52.93,NaN,NaN,NaN,Valencia,Mouratours Valencia,Salto de Obstaculos,Federación Hípica Comunidad Valenciana,Epaña,2025-04-18,2025-04-20,Nacional,68636
188622,FIN,3.0,33550,santiago escrig sánchez,163741,caretino,True,21.0,"5 años - 1,10 - Dos Fases Especial (274.2)",2025-04-18,MOURATOURS,CSNCJ,0.0,NaN,NaN,4.0,0.0,44.15,NaN,NaN,NaN,Valencia,Mouratours Valencia,Salto de Obstaculos,Federación Hípica Comunidad Valenciana,Epaña,2025-04-18,2025-04-20,Nacional,68637
188623,FIN,4.0,10366476,natalia bernal lluch,109BN94,kalachnikov gabkcal,True,21.0,"5 años - 1,10 - Dos Fases Especial (274.2)",2025-04-18,MOURATOURS,CSNCJ,0.0,NaN,NaN,4.0,0.0,44.81,NaN,NaN,NaN,Valencia,Mouratours Valencia,Salto de Obstaculos,Federación Hípica Comunidad Valenciana,Epaña,2025-04-18,2025-04-20,Nacional,68638
188624,FIN,5.0,10083655,carmen casan serneguet,170798,inca bk,True,14.0,"5 años - 1,10 - Dos Fases Especial (274.2)",2025-04-18,MOURATOURS,CSNCJ,NaN,4.0,NaN,0.0,NaN,45.86,NaN,NaN,NaN,Valencia,Mouratours Valencia,Salto de Obstaculos,Federación Hípica Comunidad Valenciana,Epaña,2025-04-18,2025-04-20,Nacional,68639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188762,FIN,18.0,60682,carmen hernández yanguas,150981,navaro,True,0.0,"7 años - 1,30 - Dos Fases Especial (274.2)",2025-04-20,MOURATOURS,CSNCJ,4.0,0.0,NaN,4.0,0.0,41.11,NaN,NaN,NaN,Valencia,Mouratours Valencia,Salto de Obstaculos,Federación Hípica Comunidad Valenciana,Epaña,2025-04-18,2025-04-20,Nacional,68777
188763,FIN,19.0,243809,sophie schwarck asmussen,169998,devil mv z,True,0.0,"7 años - 1,30 - Dos Fases Especial (274.2)",2025-04-20,MOURATOURS,CSNCJ,4.0,0.0,NaN,4.0,0.0,41.70,NaN,NaN,NaN,Valencia,Mouratours Valencia,Salto de Obstaculos,Federación Hípica Comunidad Valenciana,Epaña,2025-04-18,2025-04-20,Nacional,68778
188764,FIN,20.0,10338242,mickael weislinger,108PL89,dino f van de wolfsakker z,True,0.0,"7 años - 1,30 - Dos Fases Especial (274.2)",2025-04-20,MOURATOURS,CSNCJ,4.0,0.0,NaN,4.0,0.0,42.34,NaN,NaN,NaN,Valencia,Mouratours Valencia,Salto de Obstaculos,Federación Hípica Comunidad Valenciana,Epaña,2025-04-18,2025-04-20,Nacional,68779
188765,FIN,21.0,10056467,guillermo diez-delgado guitian,108CV73,cunegundo,True,0.0,"7 años - 1,30 - Dos Fases Especial (274.2)",2025-04-20,MOURATOURS,CSNCJ,8.0,0.0,NaN,8.0,0.0,41.47,NaN,NaN,NaN,Valencia,Mouratours Valencia,Salto de Obstaculos,Federación Hípica Comunidad Valenciana,Epaña,2025-04-18,2025-04-20,Nacional,68780


In [8]:
df_coso = df[["Concurso", "Prueba"]]
concurso_pruebas = {}
concurso_pruebas = {}
for _, row in df_coso.iterrows():  # Usar el dataframe df que proporcionaste
    concurso = row["Concurso"]
    prueba = row["Prueba"]
    if concurso in concurso_pruebas:
        if prueba not in concurso_pruebas[concurso]: # Añadido esta condición
            concurso_pruebas[concurso].append(prueba)
    else:
        concurso_pruebas[concurso] = [prueba]

concurso_pruebas

{'CSN3* Invierno': ['1,30 - A c.c. (238.2.1)',
  '1,20 - A c.c. (238.2.1)',
  '1,25 - Dos Fases Especial (274.2)',
  '1,40 - A c.c. (238.2.1)',
  '1.35 GP - Acc y desempate (238.2.2)',
  '1,30 - Dos fases Especial (Art. 274.2) (274.2)',
  '1,40 gp - A.c.c. y desempate al cronómetro (238.2.2)',
  '1,35 - Dos fases Especial (Art. 274.2) (274.2)'],
 'SPRING MET I 2025 YH*': ['6 años - 1,25 - A sin cronómetro (238.1.1)',
  '7 años - 1,35 - A sin cronómetro (238.1.1)',
  '5 años - 1,15 - A.c.c. y desempate (238.2.2)',
  '6 años - 1,25 - A.c.c. y desempate (238.2.2)',
  '7 años - 1,35 - A.c.c. y desempate (238.2.2)',
  '5 años - 1,15 - Dos Fases Especial (274.2)',
  '6 años - 1,25 - Dos Fases Especial (274.2)',
  '7 años - 1,35 - Dos Fases Especial (274.2)',
  '5 años - 1,15 - A sin cronómetro (238.1.1)'],
 'MOURA TOURS VALENCIA - SPRING TOUR': ['1,35 (G) - A c.c. (238.2.1)',
  '1,30 - A c.c. (238.2.1)',
  '1,40 (G) - DOS FASES (A c/c - A c/c) (274.1.5.3)',
  '1,45 (G) - A c.c. (238.2.1)',
 

## Inserción a BBDD

In [3]:
import psycopg2 as ps

In [4]:
import pandas as pd
from sqlalchemy import create_engine, text

# CONFIGURA TU CONEXIÓN
usuario = 'postgres'
password = 'admin'
host = 'localhost'
puerto = 5432
basedatos = 'BBDD_Hipica'

# CREA EL MOTOR DE CONEXIÓN
engine = create_engine(f'postgresql+psycopg2://{usuario}:{password}@{host}:{puerto}/{basedatos}')

In [5]:
conn = ps.connect(
    dbname = "BBDD_Hipica", # base a la que nos queremos conectar
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432" # puerto en el que s eencuentra postgres
)

In [6]:
cur = conn.cursor()

In [7]:
# COmprobamos que la conexión está creada y conectada
cur.execute("SELECT version();")
cur.fetchone() 

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

### Inserción disciplinas

In [8]:
data_insert_disciplinas = ["Salto de Obstaculos", "Concurso Completo", "Doma clasica"]


In [9]:
data_to_insert = [(disciplina,) for disciplina in data_insert_disciplinas]

In [10]:
# Creamos la query de inserción de los datos de ciudad
insert_query_disciplinas = """ 
                        INSERT INTO disciplinas(disciplina)
                        VALUES (%s) 
"""

In [19]:
conn.rollback()

In [12]:
query2 = """
    SELECT * 
    FROM disciplinas; 
"""
cur.execute(query2)
cur.fetchall()

[(1, 'Salto de Obstaculos'), (2, 'Concurso Completo'), (3, 'Doma clasica')]

In [11]:
# Ejecutamos la query para insertar las disciplinas
cur.executemany(insert_query_disciplinas, data_to_insert)
conn.commit()

### Inserción caballos

In [13]:
df_caballos = df[["Lic_caballo", "Caballo"]]

In [14]:
# Creamos la lista que va a contener toda la información 
data_insert_caballos = []

# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in df_caballos.iterrows():
    id_caballo = row["Lic_caballo"]
    nombre_caballo = row["Caballo"]

    tupla_caballo = (id_caballo, nombre_caballo)

    if tupla_caballo not in data_insert_caballos:
        data_insert_caballos.append(tupla_caballo)

In [15]:
insert_query_caballos = """ 
                        INSERT INTO caballos (id_caballo, nombre_caballo)
                        VALUES (%s, %s)
"""    

In [16]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_caballos, data_insert_caballos)
conn.commit()  

### Inserción jinetes

In [17]:
df_jinetes = df[["Lic_jinete", "Jinete"]]

In [18]:
# Creamos la lista que va a contener toda la información 
data_insert_jinetes = []

# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in df_jinetes.iterrows():
    id_jinete = row["Lic_jinete"]
    nombre_jinete = row["Jinete"]

    tupla_jinete = (id_jinete, nombre_jinete)

    if tupla_jinete not in data_insert_jinetes:
        data_insert_jinetes.append(tupla_jinete)

In [19]:
insert_query_jinetes = """ 
                        INSERT INTO jinetes (id_jinete, nombre_jinete)
                        VALUES (%s, %s)
"""

In [20]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_jinetes, data_insert_jinetes)
conn.commit()

### Inserción concursos

In [21]:
df_concursos = df[["Concurso", "Categoria", "País", "Provincia", "Localidad", "Ambito", "Federación", "Inicio", "Final"]]

In [22]:
# Creamos la lista que va a contener toda la información 
data_insert_concursos = []
# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in df_concursos.iterrows():
    nombre_concurso = row["Concurso"]
    categoria_concurso = row["Categoria"]
    pais_concurso = row["País"]
    provincia_concurso = row["Provincia"]
    localidad_concurso = row["Localidad"]
    ambito_concurso = row["Ambito"]
    federacion_concurso = row["Federación"]
    inicio_concurso = row["Inicio"]
    final_concurso = row["Final"]

    tupla_concursos = (nombre_concurso, categoria_concurso, pais_concurso, provincia_concurso, 
                       localidad_concurso, ambito_concurso, federacion_concurso, 
                       inicio_concurso, final_concurso)

    if tupla_concursos not in data_insert_concursos:
        data_insert_concursos.append(tupla_concursos)

In [23]:
insert_query_concursos = """ 
                        INSERT INTO concursos (nombre_concurso, categoria_concurso, 
                                               pais_concurso, provincia_concurso, 
                                               localidad_concurso, ambito_concurso,
                                               federacion_concurso, fecha_inicio_concurso,
                                               fecha_fin_concurso)
                        VALUES (%s, %s,  %s, %s, %s, %s, %s, %s, %s)
"""

In [24]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_concursos, data_insert_concursos)
conn.commit()


### Inserción pruebas

In [25]:
cur.execute("SELECT disciplina, id_disciplina FROM disciplinas")
dictio_disciplinas = dict(cur.fetchall())

In [26]:
df_pruebas = df[["Prueba", "Fecha_prueba", "Disciplina"]]

In [27]:
data_insert_pruebas = []

for _, row in df_pruebas.iterrows():
    nombre_prueba = row["Prueba"]
    fecha_prueba = row["Fecha_prueba"]
    disciplina_prueba = dictio_disciplinas.get(row["Disciplina"])

    tupla_pruebas = (nombre_prueba, fecha_prueba, disciplina_prueba)

    if tupla_pruebas not in data_insert_pruebas:
        data_insert_pruebas.append(tupla_pruebas)

In [28]:
insert_query_pruebas = """ 
                        INSERT INTO pruebas (nombre_prueba, fecha_prueba,id_disciplina)
                        VALUES (%s, %s,  %s)
"""

In [29]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_pruebas, data_insert_pruebas)
conn.commit()

### Inserción concursos

In [30]:
cur.execute("SELECT nombre_jinete, id_jinete FROM jinetes")
dictio_jinetes = dict(cur.fetchall())
cur.execute("SELECT nombre_caballo, id_caballo FROM caballos")
dictio_caballos = dict(cur.fetchall())
cur.execute("SELECT nombre_concurso, id_concurso FROM concursos")
dictio_concursos = dict(cur.fetchall())
cur.execute("SELECT id_prueba, nombre_prueba, fecha_prueba FROM pruebas")
coso = cur.fetchall()
df_pruebas = pd.DataFrame(coso)
df_pruebas = df_pruebas.rename(columns = {0 : "id_prueba", 1 : "Prueba", 2: "Fecha_prueba"})
df_pruebas['Fecha_prueba'] = pd.to_datetime(df_pruebas['Fecha_prueba'])


In [31]:
df_pruebas

,id_prueba,Prueba,Fecha_prueba
0,1,"1,30 - A c.c. (238.2.1)",2025-01-10
1,2,"6 años - 1,25 - A sin cronómetro (238.1.1)",2025-01-14
2,3,"1,35 (G) - A c.c. (238.2.1)",2025-01-24
3,4,"7 años - 1,35 - A sin cronómetro (238.1.1)",2025-01-14
4,5,"5 años - 1,15 - A.c.c. y desempate (238.2.2)",2025-01-15
...,...,...,...
1136,1137,"1,40 - A c.c. (238.2.1)",2025-04-26
1137,1138,1.35 GP - Acc y desempate (238.2.2),2025-04-26
1138,1139,"1,30 - Dos fases Especial (Art. 274.2) (274.2)",2025-04-27
1139,1140,"1,40 gp - A.c.c. y desempate al cronómetro (23...",2025-04-27


In [32]:
df_resultados = df[["id_resultado", "Jinete", "Caballo", "Concurso", "Prueba", "Posicion", "Estado", "Premio", "Dinero_premio", "Fecha_prueba", "Inicio", "Final"]]
df_resultados['Fecha_prueba'] = pd.to_datetime(df_resultados['Fecha_prueba'])
df_resultados['Inicio'] = pd.to_datetime(df_resultados['Inicio'])
df_resultados['Final'] = pd.to_datetime(df_resultados['Final'])

C:\Users\gabri\AppData\Local\Temp\ipykernel_9052\3065513238.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultados['Fecha_prueba'] = pd.to_datetime(df_resultados['Fecha_prueba'])
C:\Users\gabri\AppData\Local\Temp\ipykernel_9052\3065513238.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultados['Inicio'] = pd.to_datetime(df_resultados['Inicio'])
C:\Users\gabri\AppData\Local\Temp\ipykernel_9052\3065513238.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [55]:
df_resultados

,id_resultado,Jinete,Caballo,Concurso,Prueba,Posicion,Estado,Premio,Dinero_premio,Fecha_prueba,Inicio,Final
0,1,pedro miguel da silva manso,bridgetown jones os,CSN3* Invierno,"1,30 - A c.c. (238.2.1)",1,FIN,True,450.0,2025-01-10,2025-01-10,2025-01-12
1,2,ignacio astolfi lópez,quinto blue ps,CSN3* Invierno,"1,30 - A c.c. (238.2.1)",2,FIN,True,300.0,2025-01-10,2025-01-10,2025-01-12
2,3,francisco picao caldeira,flor du paso,CSN3* Invierno,"1,30 - A c.c. (238.2.1)",3,FIN,True,200.0,2025-01-10,2025-01-10,2025-01-12
3,4,jesús torres garcía,diabetto ps,CSN3* Invierno,"1,30 - A c.c. (238.2.1)",4,FIN,True,150.0,2025-01-10,2025-01-10,2025-01-12
4,5,nina fagerstrom,hasenacher cervin,CSN3* Invierno,"1,30 - A c.c. (238.2.1)",5,FIN,True,125.0,2025-01-10,2025-01-10,2025-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...
194685,72728,ignacio astolfi lópez,grupo prom jivaro,CSN3* Invierno,"1,35 - Dos fases Especial (Art. 274.2) (274.2)",23.0,FIN,True,0.0,2025-01-10,2025-01-10,2025-01-12
194686,72729,alejandro alonso buzo,houston du gast,CSN3* Invierno,"1,35 - Dos fases Especial (Art. 274.2) (274.2)",24.0,FIN,True,0.0,2025-01-10,2025-01-10,2025-01-12
194687,72730,patricio de valdenebro del rey,klimax van het lindehof,CSN3* Invierno,"1,35 - Dos fases Especial (Art. 274.2) (274.2)",NaN,RET,True,0.0,2025-01-10,2025-01-10,2025-01-12
194688,72731,francisco román cantillana,uloubet de vanau,CSN3* Invierno,"1,35 - Dos fases Especial (Art. 274.2) (274.2)",NaN,NOP,True,0.0,2025-01-10,2025-01-10,2025-01-12


In [33]:
df_merged = pd.merge(df_resultados, df_pruebas, on='Prueba', suffixes=('_resultados', '_pruebas'), how='left')
# Filtrar donde la Fecha_prueba del df_pruebas esté dentro del rango [Inicio, Final]
df_filtered = df_merged[
    (df_merged['Fecha_prueba_pruebas'] >= df_merged['Inicio']) & (df_merged['Fecha_prueba_pruebas'] <= df_merged['Final'])
]
df_final = df_filtered.groupby('id_resultado').first().reset_index()
# Unir esta información de vuelta al DataFrame original de resultados
df_resultados_bueno = pd.merge(df_resultados, df_final[['id_resultado', 'id_prueba']], on='id_resultado', how='left')
df_resultados_bueno = df_resultados_bueno.drop(columns = ["Prueba", "Fecha_prueba", "Inicio", "Final"], axis = 1)
df_resultados_bueno

,id_resultado,Jinete,Caballo,Concurso,Posicion,Estado,Premio,Dinero_premio,id_prueba
0,1,pedro miguel da silva manso,bridgetown jones os,CSN3* Invierno,1,FIN,True,450.0,1
1,2,ignacio astolfi lópez,quinto blue ps,CSN3* Invierno,2,FIN,True,300.0,1
2,3,francisco picao caldeira,flor du paso,CSN3* Invierno,3,FIN,True,200.0,1
3,4,jesús torres garcía,diabetto ps,CSN3* Invierno,4,FIN,True,150.0,1
4,5,nina fagerstrom,hasenacher cervin,CSN3* Invierno,5,FIN,True,125.0,1
...,...,...,...,...,...,...,...,...,...
72727,72728,ignacio astolfi lópez,grupo prom jivaro,CSN3* Invierno,23.0,FIN,True,0.0,1141
72728,72729,alejandro alonso buzo,houston du gast,CSN3* Invierno,24.0,FIN,True,0.0,1141
72729,72730,patricio de valdenebro del rey,klimax van het lindehof,CSN3* Invierno,NaN,RET,True,0.0,1141
72730,72731,francisco román cantillana,uloubet de vanau,CSN3* Invierno,NaN,NOP,True,0.0,1141


In [34]:
data_insert_resultados = []
lista = ["ELI", "NOP", "RET", "DSQ", "ret", "nop", "dsq", "eli"]

for _, row in df_resultados_bueno.iterrows():
    
    id_resultado = int(row["id_resultado"])
    id_jinete = dictio_jinetes.get(row["Jinete"])
    id_caballo = dictio_caballos.get(row["Caballo"])
    id_concurso = dictio_concursos.get(row["Concurso"])
    id_prueba = row["id_prueba"]
    # puesto debe ser None si es NaN o está en lista
    puesto = row["Posicion"] if pd.notna(row["Posicion"]) and row["Posicion"] not in lista else None
    estado = row["Estado"]
    premio = row["Premio"]
    dinero_premio = row["Dinero_premio"]

    tupla_resultados = (id_resultado, id_jinete, id_caballo, id_prueba, id_concurso, puesto, estado, premio, dinero_premio)

    if tupla_resultados not in data_insert_resultados:
        data_insert_resultados.append(tupla_resultados)

In [35]:
len(data_insert_resultados)

72732

In [36]:
insert_query_resultados = """ 
                        INSERT INTO resultados (id_resultado, id_jinete, id_caballo, id_prueba, id_concurso, puesto, estado, premio, dinero_premio)
                        VALUES (%s, %s, %s,  %s, %s, %s, %s, %s, %s)
"""

In [37]:
# Ejecutamos la query y le indicamos los valores 

cur.executemany(insert_query_resultados, data_insert_resultados)
conn.commit()

In [37]:
dictio_pruebas

{('1,30 - A c.c. (238.2.1)', datetime.date(2025, 1, 10)): 1,
 ('6 años - 1,25 - A sin cronómetro (238.1.1)', datetime.date(2025, 1, 14)): 2,
 ('1,35 (G) - A c.c. (238.2.1)', datetime.date(2025, 1, 24)): 3,
 ('7 años - 1,35 - A sin cronómetro (238.1.1)', datetime.date(2025, 1, 14)): 4,
 ('5 años - 1,15 - A.c.c. y desempate (238.2.2)',
  datetime.date(2025, 1, 15)): 5,
 ('6 años - 1,25 - A.c.c. y desempate (238.2.2)',
  datetime.date(2025, 1, 15)): 6,
 ('7 años - 1,35 - A.c.c. y desempate (238.2.2)',
  datetime.date(2025, 1, 15)): 7,
 ('5 años - 1,15 - Dos Fases Especial (274.2)', datetime.date(2025, 1, 16)): 8,
 ('6 años - 1,25 - Dos Fases Especial (274.2)', datetime.date(2025, 1, 16)): 9,
 ('7 años - 1,35 - Dos Fases Especial (274.2)',
  datetime.date(2025, 1, 16)): 10,
 ('1,25 - A c.c. (238.2.1)', datetime.date(2025, 1, 16)): 11,
 ('1,35 (G) - A c.c. (238.2.1)', datetime.date(2025, 1, 16)): 12,
 ('1,20 - A c.c. (238.2.1)', datetime.date(2025, 1, 10)): 13,
 ('1,25 - Dos Fases (274.1.5.

### Inserción resultados salto

In [38]:
df_resultados_salto = df[["Disciplina","id_resultado", "Ptos_obs_R1", "Ptos_tiempo_R1", "Tiempo_R1", "Ptos_obs_R2", "Ptos_tiempo_R2", "Tiempo_R2", "Ptos_obs_R3", "Ptos_tiempo_R3", "Tiempo_R3"]]

In [39]:
cur.execute("SELECT id_resultado FROM resultados")
resultados_existentes = {row[0]: None for row in cur.fetchall()}

In [40]:
data_insert_resultados_salto = []


for _, row in df_resultados_salto.iterrows():
    id_resultado = row["id_resultado"]
    if id_resultado not in resultados_existentes:
        print(f"Error: El id_resultado {id_resultado} del dataframe no existe en la tabla resultados")
        continue 

    id_disciplina = dictio_disciplinas.get(row["Disciplina"])

    ptos_obs_r1 = row["Ptos_obs_R1"] if pd.notna(row["Ptos_obs_R1"]) else None
    ptos_tmp_r1 = row["Ptos_tiempo_R1"] if pd.notna(row["Ptos_tiempo_R1"]) else None
    tiempo_r1 = row["Tiempo_R1"] if pd.notna(row["Tiempo_R1"]) else None

    ptos_obs_r2 = row["Ptos_obs_R2"] if pd.notna(row["Ptos_obs_R2"]) else None
    ptos_tmp_r2 = row["Ptos_tiempo_R2"] if pd.notna(row["Ptos_tiempo_R2"]) else None
    tiempo_r2 = row["Tiempo_R2"] if pd.notna(row["Tiempo_R2"]) else None

    ptos_obs_r3 = row["Ptos_obs_R3"] if pd.notna(row["Ptos_obs_R3"]) else None
    ptos_tmp_r3 = row["Ptos_tiempo_R3"] if pd.notna(row["Ptos_tiempo_R3"]) else None
    tiempo_r3 = row["Tiempo_R3"] if pd.notna(row["Tiempo_R3"]) else None


    tupla_resultados_salto = (id_resultado, id_disciplina, ptos_obs_r1, ptos_tmp_r1, tiempo_r1, ptos_obs_r2, ptos_tmp_r2, tiempo_r2, ptos_obs_r3, ptos_tmp_r3, tiempo_r3)

    if tupla_resultados_salto not in data_insert_resultados_salto:
        data_insert_resultados_salto.append(tupla_resultados_salto)

In [41]:

insert_query_resultados_salto = """ 
                        INSERT INTO resultados_salto (id_resultado, id_disciplina, puntos_obs_r1, puntos_tmp_r1, tiempo_r1, puntos_obs_r2, puntos_tmp_r2, tiempo_r2, puntos_obs_r3, puntos_tmp_r3, tiempo_r3)
                        VALUES (%s, %s, %s,  %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [42]:


cur.executemany(insert_query_resultados_salto, data_insert_resultados_salto)
conn.commit()

In [53]:
conn.rollback()

In [58]:
query1 = """
        SELECT * 
        FROM resultados r
            JOIN concursos c ON r.id_concurso = c.id_concurso
            JOIN pruebas p ON r.id_prueba = p.id_prueba
        WHERE c.nombre_concurso = 'MOURATOURS';
"""
cur.execute(query1)
cosita = pd.DataFrame(cur.fetchall())
cosita[20].unique()

array(['5 años - 1,10 - Dos Fases Especial (274.2)',
       '6 años - 1,20 - Dos Fases Especial (274.2)',
       '7 años - 1,30 - Dos Fases Especial (274.2)',
       '5 años - 1,10 - A c.c. (238.2.1)',
       '6 años - 1,20 - A c.c. (238.2.1)',
       '7 años - 1,30 - A c.c. (238.2.1)'], dtype=object)